

Your groups have been assigned by Nationwide, and now it is time to begin working together to build towards your Capstone presentations! Your first group project will be to gather the additional county auditor data, clean it, merge it, and begin to understand it for those counties that are considered part of the Columbus Ohio MSA: Franklin, Fairfield, Licking, Delaware, Hocking, Madison, Morrow, Perry, Pickaway, and Union. We have already worked with Franklin, Fairfield, and Licking County auditor datasets during Unit 2, and will continue working with these datasets, but your job as a group is to combine all pf the county datasets together.

A copy of auditor data for all of these counties is attached to this entry on Blackboard. Note that Franiklin, Fairfield, and Licking county data is not included since that data was already made available in Unit 2 Exercises. Additionally, Union County data is unavailable at this time - Bonus points to any team who can figure out how to get it!

Group Project 1 presentations will be at the beginning of class on June 20, and are to be 10 minutes or less in length. Presentations for this first group project can be done by one ore more members of the group. You will be graded on the following for this first group presentation:

    Presentation of basic statistics and charts showing how your team chose to sample (or not sample) data files, what format you stored it in, and any interesting facts you found while reviewing basic statistics about the data.
    What your team would like to study next about the data.
    Presentation succinctness: less than 10min in duration, highlight main points, highlight any key questions or concerns about the data that your team had as you performed the work.

Remember that this data can and should be used by your group during your Capstone work, so take your time building it well. This is your team's chance to perform its first practice run at presenting to an audience - enjoy it! The pressure to have robust data and impressive data visualization and modeling techniques can wait until your Capstone.


In [14]:
# Sampling functions
import linecache
import random
import samplingu2 as sampling 
import pandas as pd

###
### Delaware County
###

# pandas settings
pd.set_option('display.max_column', 200)

# load full dataset for initial analysis. Delaware uses "|" as a delimiter
delaware = pd.read_csv('../data/county_auditor/OH-Delaware/governmaxextract.txt', delimiter="|", low_memory=False)

# sample the data (default is 10% of all rows)
#sampling.sample_file('../data/county_auditor/OH-Delaware/governmaxextract.txt', 
#            '../data/county_auditor/OH-Delaware/delaware_county_subset.txt')

# Filter Data
# Delaware as column called Parcel_Type, but all the values in the column are 0.
# Instead the column called mClassificationId contains Land Use Codes, which we use as a filter.
# I'm assuming the data of interest is Residental Property. According to http://codes.ohio.gov/oac/5703-25-10
# codes between 500 and 599 are Taxable residential real property
delaware_subset = delaware[delaware.mClassificationId >= 500]
delaware_subset = delaware_subset[delaware_subset.mClassificationId <= 599]

# Filter out sales above 1.5 million dollars and sales of zero dollars
delaware_subset = delaware_subset[delaware_subset.SaleAmount < 1500000]
delaware_subset = delaware_subset[delaware_subset.SaleAmount > 0]

# Delaware separates out the address across six columns. Combine them into one Street Address
street_address = delaware_subset.mlocStrDir.fillna('') + " "
street_address += delaware_subset.mlocStrNo.fillna('') + " "
street_address += delaware_subset.mlocStrNo2.fillna('') + " "
street_address += delaware_subset.mlocStrName.fillna('') + " "
street_address += delaware_subset.mlocStrSuffix.fillna('') + " "
street_address += delaware_subset.mlocStrSuffixDir.fillna('')

# Add the street_address to the dataframe at index 2
delaware_subset.insert(2, "StreetAddress", street_address)

#restrict the columns of interest
delaware_columns = [ 'mpropertyNumber', 'mlegalDescription','StreetAddress', 'mlocCity','mlocState','mlocZipCode',
                     'mlocDescription','mClassificationId','macres','mfrontfootage', 'OwnName', 
                     'mtaxsetCode','mstateSchoolcode','MKT_Land_Value','MKT_Impr_Value',
                     'CAUV_Value','MKT_Total_Value','MKT_Tot_Total','SaleDate','SaleAmount',
                     'Conveyance','Hmstd_Flag','LandVal','ImproveVal','SpecAssessment',
                     'AnnualTax','TresCode']

# Apply the column masks to the dataframe
delaware_subset = delaware_subset[delaware_columns].copy()

delaware_subset.rename(
    {'mpropertyNumber' : 'ParcelNumber',
     'mlegalDescription' : 'LegalDescription',
     'mlocCity' : 'City',
     'mlocState' : 'State',
     'mlocZipCode' : 'ZipCode',
     'mlocDescription' : 'Description',
     'mClassificationId' : 'LandUseCode',
     'macres' : 'Acres',
     'mfrontfootage' : 'FrontFootage',
     'OwnName' : 'Owner',
     'mtaxsetCode' : 'TaxSetCode',
     'mstateSchoolcode' : 'SchoolDistrict',
     'MKT_Land_Value' : 'MarketLandValue',
     'MKT_Impr_Value' : 'MarketImprovementValue',
     'CAUV_Value' : 'CAUV',
     'MKT_Total_Value' : 'MarketValue',
     'MKT_Tot_Total' : 'MarketTotal',
     'SaleAmount' : 'SalePrice',
     'Conveyance' : 'Conveyance',
     'Hmstd_Flag' : 'HomesteadFlag',
     'LandVal' : 'LandValue',
     'ImproveVal' : 'ImprovementValue'
    },
    axis=1,
    inplace=True
)

for column in delaware_subset.columns:
    print(column)

ParcelNumber
LegalDescription
StreetAddress
City
State
ZipCode
Description
LandUseCode
Acres
FrontFootage
Owner
TaxSetCode
SchoolDistrict
MarketLandValue
MarketImprovementValue
CAUV
MarketValue
MarketTotal
SaleDate
SalePrice
Conveyance
HomesteadFlag
LandValue
ImprovementValue
SpecAssessment
AnnualTax
TresCode


In [12]:
### Sample County Data

# using sampleCounty in samplingsu2.py
delaware_subset = sampling.sampleCounty(delaware_subset, 0.1)

display(delaware_subset.head())

,ParcelNumber,LegalDescription,StreetAddress,City,State,ZipCode,Description,LandUseCode,Acres,FrontFootage,Owner,TaxSetCode,SchoolDistrict,MarketLandValue,MarketImprovementValue,CAUV,MarketValue,MarketTotal,SaleDate,SalePrice,Conveyance,HomesteadFlag,LandValue,ImprovementValue,SpecAssessment,AnnualTax,TresCode
80690,600-341-16-008-000,LOT 1552 ^MUIRFIELD PH 33 ...,9208 LERWICK DR,DUBLIN,OH,43017,NaN,510.0,0.000,0.0,RAUDALES RAUL A JR,10.0,2513.0,85000,474700,0,559700,559700,02-15-2018,548300,511,N,29750,166150,1,13276.09,TU1213
8793,317-314-04-185-000,LOT 5397 ^SHERBROOK PH8 ...,6163 HEMINGWAY PL,WESTERVILLE,OH,43082,NaN,510.0,0.306,0.0,SCHWIND DONALD E,17.0,2514.0,50000,185600,0,235600,235600,02-14-2002,217875,682,N,17500,64960,1,5469.42,TU0604
81873,600-426-01-046-000,LOTS 59 ^ ...,HARRIOTT DR,POWELL,OH,43065,NaN,500.0,0.000,0.0,LEAR GILBERT M,9.0,2513.0,5000,0,0,5000,5000,11-10-2010,200000,3248,N,1750,0,0,112.74,TU1213
56034,419-121-02-009-000,LOTS 2613 ^ ...,60 WOOTRING ST,DELAWARE,OH,43015,NaN,510.0,0.118,0.0,BENTLEY MELANIE,14.0,2103.0,16700,85700,0,102400,102400,05-04-2017,102000,1566,Y,5850,30000,0,1692.22,P 6-03
30485,318-434-01-011-400,UNIT 294^VILLAGE AT POLARIS PARK CONDOS 14TH A...,1562 SARGAS ST,COLUMBUS,OH,43240,NaN,550.0,0.000,0.0,STANLEY DIMITRIOUS,46.0,2104.0,30000,143100,0,173100,173100,07-09-2004,167076,3554,N,10500,50090,0,4035.74,TU1213


<bound method IndexOpsMixin.tolist of Index(['ParcelNumber', 'LegalDescription', 'StreetAddress', 'City', 'State',
       'ZipCode', 'Description', 'LandUseCode', 'Acres', 'FrontFootage',
       'Owner', 'TaxSetCode', 'SchoolDistrict', 'MarketLandValue',
       'MarketImprovementValue', 'CAUV', 'MarketValue', 'MarketTotal',
       'SaleDate', 'SalePrice', 'Conveyance', 'HomesteadFlag', 'LandValue',
       'ImprovementValue', 'SpecAssessment', 'AnnualTax', 'TresCode'],
      dtype='object')>


In [17]:
###
### Hocking County
###

hocking = pd.read_csv('../data/county_auditor/OH-Hocking/Grid_salef_2019-03-20.csv', delimiter=",", low_memory=False)
hocking_subset = hocking

# Function to change the values in a field
def rename_values(row, column, original, replacement):
    field = row[column]
    return field.replace(original, replacement).strip()

### Filters and Drops

# filter out any property that has NaN Sqft, House Number (Number) is NaN (Assume no house on property)
hocking_subset = hocking_subset[pd.notna(hocking_subset.Sqft)]
hocking_subset = hocking_subset[pd.notna(hocking_subset.Number)]

# drop
hocking_subset.drop(['Corp / Village'], axis=1, inplace=True)
hocking_subset.drop(['MHRE'], axis=1, inplace=True)

### Bathrooms

# Combine FullBaths and HalfBaths to Bathrooms
bathrooms = (hocking_subset.FullBaths.fillna(0) + 0.5 * hocking_subset.HalfBaths.fillna(0))
hocking_subset.insert(hocking_subset.columns.get_loc('HalfBaths') - 1, "Bathrooms", bathrooms)

# Remove FullBaths, HalfBaths
hocking_subset.drop(['FullBaths'], axis=1, inplace=True)
hocking_subset.drop(['HalfBaths'], axis=1, inplace=True)

### Garage

# Rename Type to GarageType
hocking_subset.rename({'Type' : 'GarageType'}, axis=1, inplace=True)
hocking_subset.loc[:,'GarageType'] = hocking_subset.GarageType.fillna('None')

# Rename D to Detached and A to Attached
hocking_subset.loc[:,'GarageType'] = hocking_subset.apply(rename_values, args=('GarageType','D','Detached'), axis=1)
hocking_subset.loc[:,'GarageType'] = hocking_subset.apply(rename_values, args=('GarageType','A','Attached'), axis=1)

# GarageSqft: If NaN, then zero.
hocking_subset.loc[:, 'GarageSqft'] = hocking_subset['GarageSqft'].fillna(0)

### PropClass aka Land Use Codes

# PropClass is Land Use Codes; only include Residental Properties
hocking_subset = hocking_subset[hocking_subset.PropClass >= 500]
hocking_subset = hocking_subset[hocking_subset.PropClass <= 599]

### Basement

# Basement: Convert NaN to 'No', B to 'Full' and H to 'Half'
hocking_subset.loc[:,'Basement'] = hocking_subset.Basement.fillna('No')
hocking_subset.loc[:,'Basement'] = hocking_subset.apply(rename_values, args=('Basement','B','Full'), axis=1)
hocking_subset.loc[:,'Basement'] = hocking_subset.apply(rename_values, args=('Basement','H','Half'), axis=1)

# Basement Sqrt Footage: If NaN, replace with zero
hocking_subset.loc[:,'BasementSqft'] = hocking_subset.BasementSqft.fillna('0')

# Basement Finished: If NaN, replace with No. Replace F with Finished
hocking_subset.loc[:, 'Fin Base'] = hocking_subset['Fin Base'].fillna('No')
hocking_subset.loc[:,'Fin Base'] = hocking_subset.apply(rename_values, args=('Fin Base','F','Finished'), axis=1)

# Bedrooms: If NaN, then 1
hocking_subset.loc[:, 'Bedrooms'] = hocking_subset['Bedrooms'].fillna(1)

### Air

# Air "Air Condition": If NaN, then No. If Y, then change to Yes
hocking_subset.loc[:, 'Air'] = hocking_subset['Air'].fillna('No')
hocking_subset.loc[:,'Air'] = hocking_subset.apply(rename_values, args=('Air','Y','Yes'), axis=1)


### Roofing

# Roof Type: For each RoofType as NaN, then RoofType is assigned to the highest type when Roof and RoofType are group.
# Use:
# roof_roofType = hocking_subset.groupby(['Roof', 'RoofType'])['Yr Built'].count()
# display(roof_roofType)
# to find the most common RoofType for each roof

#hocking_subset['Roof'] = hocking_subset.Roof.astype(object)
hocking_subset.loc[:, 'Roof'] = hocking_subset['Roof'].fillna('O')
hocking_subset.loc[:, 'Roof'] = hocking_subset.apply(rename_values, args=('Roof','`','O'), axis=1)

# Name NaN as unknown
hocking_subset.loc[:, 'RoofType'] = hocking_subset['RoofType'].fillna('unknown')

# Replace unknow with common RoofType
for index, row in hocking_subset.iterrows():
    if isinstance(row.RoofType, str):
        roof = row['Roof']
        rtype = row['RoofType']
        if rtype == 'unknown':
            if roof == 'M':
                new_value = row.RoofType.replace('unknown','MET')
                hocking_subset.loc[index, 'RoofType'] = new_value
            if roof == 'S':
                new_value = row.RoofType.replace('unknown','ASP')
                hocking_subset.loc[index, 'RoofType'] = new_value
            if roof == 'O':
                new_value = row.RoofType.replace('unknown','ROL')
                hocking_subset.loc[index, 'RoofType'] = new_value
            if roof == 'L':
                new_value = row.RoofType.replace('unknown','SLA')
                hocking_subset.loc[index, 'RoofType'] = new_value
                
### Rooms

# If Room is NaN, then set Median room
# Use:
#hocking_subset.loc[:, 'Rooms'] = hocking_subset['Rooms'].fillna('unknown')
#rooms_saleprice = hocking_subset.groupby(['Rooms', 'Yr Built'])['Sale Price'].median()
#display(rooms_saleprice)
# To determine median number of rooms

hocking_subset.loc[:, 'Rooms'] = hocking_subset['Rooms'].fillna(5)


### Rename Columns

# Remove spaces in names and clarify names
hocking_subset.rename(
    {
     'Parcel Number' : 'ParcelNumber',
     'School District' : 'SchoolDistrict',
     'Sale #': 'SaleNumber',
     'Sale Price' : 'SalePrice',
     'Yr Built' : 'YearBuilt',
     'Fin Base' : 'FinishedBasement',
     'Heat Type' : 'HeatType',
     'Property Card' : 'PropertyCard'
    },
    axis=1,
    inplace=True
)

for column in hocking_subset.columns:
    print(column)

ParcelNumber
Number
Street
SchoolDistrict
Township
SaleNumber
Month
Year
SalePrice
Sqft
Acres
YearBuilt
Story
Basement
Frame
Bedrooms
BasementSqft
FinishedBasement
Air
GarageSqft
GarageType
HeatType
Roof
RoofType
Rooms
Bathrooms
Grade
NeighCode
PropClass
LandValue
BldgValue
PropertyCard


In [27]:
### Sample County Data

# using sampleCounty in samplingsu2.py
hocking_subset = sampling.sampleCounty(hocking_subset, 0.1)

display(hocking_subset.head())

,ParcelNumber,Number,Street,SchoolDistrict,Township,SaleNumber,Month,Year,SalePrice,Sqft,Acres,YearBuilt,Story,Basement,Frame,Bedrooms,BasementSqft,FinishedBasement,Air,GarageSqft,GarageType,HeatType,Roof,RoofType,Rooms,Bathrooms,Grade,NeighCode,PropClass,LandValue,BldgValue,PropertyCard
7294,160000290000,20223.0,ST RT 278,2A,4L,128.0,3,7,160000,1796.0,7500.0,63/A,1,Full,F,1.0,1316,No,Yes,0.0,None,GAS,S,GAB,6.0,2.0,D+5,7165.0,511,12600.0,71120.0,http://www.realestate.co.hocking.oh.us/cards/C...
2706,80001730200,26616.0,WILD CAT,2A,4G,559.0,11,14,25000,1960.0,50000.0,NaN,NaN,No,NaN,1.0,1176,Finished,Yes,0.0,None,CENTRAL,M,MET,7.0,2.5,NaN,7085.0,571,26640.0,NaN,http://www.realestate.co.hocking.oh.us/cards/C...
3421,210000570600,22430.0,BUNKER HILL,2C,4D,547.0,10,15,232700,1902.0,50000.0,2001,1,Full,F,3.0,1344,No,Yes,480.0,Attached,PROPANE,S,ASP,6.0,2.0,C+10,7215.0,511,22840.0,174430.0,http://www.realestate.co.hocking.oh.us/cards/C...
1782,40002180000,85.0,MULBERRY STS,2A,4B,600.0,11,13,60000,2378.0,NaN,1890,2,Full,F,1.0,257,No,No,348.0,Attached,WARM AIR,L,SLA,7.0,3.0,C-10,434.0,530,22560.0,41200.0,http://www.realestate.co.hocking.oh.us/cards/C...
646,20026080200,10033.0,COAKLEY,2A,4B,400.0,9,11,53000,936.0,4200.0,2001,1,No,F,2.0,0,No,Yes,0.0,None,CENTRAL,S,ASP,6.0,1.0,D+10,7025.0,511,13440.0,79910.0,http://www.realestate.co.hocking.oh.us/cards/C...


In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../data/output.sqlite')

delaware_subset.to_sql("Delaware", con=engine, if_exists='replace')
hocking_subset.to_sql("Hocking", con=engine, if_exists='replace')